In [10]:
import pyautogui
import cv2
import time
import os

# Esperar 5 segundos para prepararte
time.sleep(5)

# Captura de pantalla
captura = pyautogui.screenshot()
captura_path = "captura_temporal.png"
captura.save(captura_path)

# Ruta de la imagen plantilla
plantilla_path = "Mendeley_icon.png"
imagen_a_buscar = cv2.imread(plantilla_path)

# Verificar que la imagen fue cargada correctamente
if imagen_a_buscar is None:
    print("❌ No se pudo cargar la imagen 'Mendeley_icon.png'.")
    exit()

# Leer la captura de pantalla
captura_img = cv2.imread(captura_path)

# Convertir ambas a escala de grises
captura_gray = cv2.cvtColor(captura_img, cv2.COLOR_BGR2GRAY)
plantilla_gray = cv2.cvtColor(imagen_a_buscar, cv2.COLOR_BGR2GRAY)

# Búsqueda por template matching
resultado = cv2.matchTemplate(captura_gray, plantilla_gray, cv2.TM_CCOEFF_NORMED)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(resultado)

# Mostrar información en consola
print(f"🧪 Máxima coincidencia: {max_val:.4f}")
print(f"📍 Coordenadas encontradas (imagen): {max_loc}")

# Umbral de coincidencia
umbral = 0.8

if max_val >= umbral:
    alto, ancho = plantilla_gray.shape

    # Escalar coordenadas si la imagen es retina (usualmente el doble)
    screen_w, screen_h = pyautogui.size()
    factor_escalado = screen_w / captura_gray.shape[1]

    centro_x = int((max_loc[0] + ancho // 2) * factor_escalado)
    centro_y = int((max_loc[1] + alto // 2) * factor_escalado)

    print(f"🖥️ Resolución de pantalla: {screen_w}x{screen_h}")
    print(f"🎯 Coordenadas a mover (escaladas): ({centro_x}, {centro_y})")

    # Validar que esté dentro de la pantalla
    if 0 <= centro_x <= screen_w and 0 <= centro_y <= screen_h:
        pyautogui.moveTo(centro_x, centro_y, duration=2, tween=pyautogui.easeOutQuad)
        pyautogui.click(clicks=2, interval=0.2)
        print("✅ Imagen encontrada y doble clic realizado.")
    else:
        print("⚠️ Coordenadas fuera de la pantalla. Escalado incorrecto.")

    # Dibujar el rectángulo
    top_left = max_loc
    bottom_right = (top_left[0] + ancho, top_left[1] + alto)
    cv2.rectangle(captura_img, top_left, bottom_right, (0, 255, 0), 2)
    cv2.imwrite("resultado_debug.png", captura_img)
    print("🖼️ Se ha guardado 'resultado_debug.png' con el área marcada.")

else:
    print("❌ Imagen no encontrada con suficiente coincidencia.")

# Borrar la captura temporal
os.remove(captura_path)


🧪 Máxima coincidencia: 0.9997
📍 Coordenadas encontradas (imagen): (1378, 806)
🖥️ Resolución de pantalla: 1440x900
🎯 Coordenadas a mover (escaladas): (719, 432)
✅ Imagen encontrada y doble clic realizado.
🖼️ Se ha guardado 'resultado_debug.png' con el área marcada.
